# Higher energy states SSVQE with 4 qubits (Algorithm C)

### Baseline recommendation: 12 entanglement layers, Maxiter = 3500, NUM_SHOTS = 3000, SPSA algorithm --> gives multipliers of (2.30, 4.19, 12.84) for first three energy states

Based off v3.7 (baseline three and four qubits) with SPSA optimizer and Subspace VQE algorithm

Four and three qubit expanded case with circuit from “Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets” paper (2017). This implements entanglment between the qubit lines using RZX gates.

The mathematics of the lattice and Fourier transform block are referenced from "Quantum Computation and Visualization of Hamiltonians Using Discrete Quantum Mechanics and IBM Qiskit" by Raffaele Miceli and Michael McGuigan (see PHYS 470 report for more details)


## 1. Setup and constants

In [ ]:
import numpy as np 
import scipy
from scipy.optimize import minimize, BFGS
import time
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import Gate
from qiskit import Aer, transpile, assemble
from qiskit.quantum_info.operators import Operator, Pauli    # for operator
from qiskit.aqua.algorithms import NumPyEigensolver
from qiskit import Aer, transpile, assemble      # to run quantum simulation
backend = Aer.get_backend("qasm_simulator")
from qiskit.aqua.components.optimizers import COBYLA    # for optimization

np.set_printoptions(precision=7)    # numpy print precision
np.set_printoptions(suppress=True)  # suppress scientific notation

# define constants
QUBIT_COUNT = 3
DEPTH = 3                     # depth of ansatz
PARAM_COUNT = QUBIT_COUNT*(2*DEPTH+2)   
NUM_SHOTS = 3000              # number of shots per quantum simulation
ENERGY_STATES = 1             # which states to consider. Set 1 for ground, 2 for second .etc

np.random.seed(1)
wild_params = np.random.rand(PARAM_COUNT)
#print("The starting random ansatz parameters are:", wild_params)

In [ ]:
# Making Hamiltonian (from Miceli and McGuigan paper)
n = 2**QUBIT_COUNT    # for code clarity, in matrix preparation
lattice = []
leftmost = (1-n)/2    # consider lattice 15 June
Xpos = []     # Position matrix

for i in range(n):
    lattice.append(leftmost + i)
    # for position matrix    
    Xpos_row = []
    for j in range(n):
        Xpos_row.append(0)
    Xpos_row[i] = (leftmost + i)*np.sqrt(2*np.pi/n)
    Xpos.append(Xpos_row)
    #print("Xpos_row is: ",Xpos_row)
    
print("lattice is: ", lattice)
Xpos = np.array(Xpos)
#print("Xpos is: ")
#print(Xpos)

In [ ]:
F = np.zeros([n,n], dtype=complex)  # zero matrix 
for j in range(n):
    for k in range(n):
        x = 1/np.sqrt(n)*np.e**(2*np.pi*(1j/n)*lattice[j]*lattice[k])
        F[j][k] = x

print("Check if Fourier operator calculated is unitary (F_dagger*F=I)")
F_dagger = np.transpose(np.conjugate(F))
#print(F.dot(F_dagger))
FourierOperator = Operator(F)

P = (F_dagger.dot(Xpos.dot(F)))
Hamiltonian = P.dot(P)/2  # flat potential well model here
Hamiltonian2 = P.dot(P)/2 + Xpos.dot(Xpos)/2    # with harmonic

#print("Momentum Operator (P) :")
#print(P) 
#print("Hamiltonian (H) :")
#print(Hamiltonian)

H_dagger2 = np.transpose(np.conjugate(Hamiltonian2))
#print("H2 dagger minus H2 (should be zero)")    # check Hermitian by taking conjugate, H=H^t
#print(H_dagger2-Hamiltonian2)

In [ ]:
# get eigenvalue in numpy
from numpy import linalg as LA
print("The eigenvalues are: ", LA.eigvals(Hamiltonian))

min_eigenvalue = np.real(min(LA.eigvals(Hamiltonian)))
print("Lowest eigenvalue is ", min_eigenvalue)

## 2. Variational ansatz functions (4*(number of qubits) parameters with single rotation gate method)  (updated for VQE)

There is an entangling part and a rotation part for the qubits.

### Ansatz circuit with orthogonal state input

Set a Pauli X gate(s) at the start of ansatz for the input, to change from 000 only (000, 001, 010, 011, 100, 101, 110, 111) 
Define as function with another argument from params

In [ ]:
def get_ansatz_pos(params, input_line):    
    """
    Edited for subspace VQE 
    New ansatz builder function using DEPTH setting
    params is a list, input_line is an integer (0 to 7)
    This ansatz measures in the X basis (position space)
    """
        
    qr = QuantumRegister(QUBIT_COUNT)        
    cr = ClassicalRegister(QUBIT_COUNT)
    qc = QuantumCircuit(qr, cr)
    
    # input section
    if input_line == 0:       # 000 in computational basis
        pass
    elif input_line == 1:     # 001
        qc.rx(np.pi, qr[0])
    elif input_line == 2:     # 010
        qc.rx(np.pi, qr[1])
    elif input_line == 3:     # 011
        qc.rx(np.pi, qr[0])
        qc.rx(np.pi, qr[1])
    elif input_line == 4:     # 100
        qc.rx(np.pi, qr[2])    
    elif input_line == 5:     # 101
        qc.rx(np.pi, qr[0])
        qc.rx(np.pi, qr[2]) 
    elif input_line == 6:     # 110
        qc.rx(np.pi, qr[1])
        qc.rx(np.pi, qr[2]) 
    elif input_line == 7:     # 111
        qc.rx(np.pi, qr[0])
        qc.rx(np.pi, qr[1])
        qc.rx(np.pi, qr[2])
    else:
        print("Error: input_line should be integer from 0 to 7")
        return
    
    qc.barrier()    # for clarity, after input setup
    
    for layer in range(DEPTH):    
        # pre-entangling section
        qc.ry(params[2*layer], qr[0])    
        qc.rz(params[2*layer + 1], qr[0])
        qc.ry(params[2*layer + 2*DEPTH + 2], qr[1])   # second wire
        qc.rz(params[2*layer + 2*DEPTH + 3], qr[1])    
        if QUBIT_COUNT >= 3:
            qc.ry(params[2*layer + 4*DEPTH + 4], qr[2])   # third wire
            qc.rz(params[2*layer + 4*DEPTH + 5], qr[2])    
        if QUBIT_COUNT >= 4:
            qc.ry(params[2*layer + 6*DEPTH + 6], qr[3])   # fourth wire     
            qc.rz(params[2*layer + 6*DEPTH + 7], qr[3])

        # entangling
        qc.rzx(np.pi/2, qr[0], qr[1])
        if QUBIT_COUNT >= 3:
            qc.rzx(np.pi/2, qr[1], qr[2])
        if QUBIT_COUNT >= 4:
            qc.rzx(np.pi/2, qr[2], qr[3])
        qc.barrier()    
    
    qc.ry(params[2*DEPTH], qr[0])   # first wire 
    qc.rz(params[2*DEPTH+1], qr[0])
    qc.ry(params[4*DEPTH+2], qr[1])   # second wire
    qc.rz(params[4*DEPTH+3], qr[1])
    if QUBIT_COUNT >= 3:
        qc.ry(params[6*DEPTH+4], qr[2])  # third wire
        qc.rz(params[6*DEPTH+5], qr[2])    
    if QUBIT_COUNT >= 4:
        qc.ry(params[8*DEPTH+6], qr[3])   # fourth wire      
        qc.rz(params[8*DEPTH+7], qr[3])
    
    for i in range(QUBIT_COUNT):
        qc.measure(qr[i], cr[i])

    return qc

In [ ]:
print("===== New get_ansatz_pos circuit =====")
my_circuit = get_ansatz_pos(wild_params, 3)
#my_circuit.draw(output='mpl')

### New momentum Subspace Search VQE circuit for subspace search

with the input state modification

In [ ]:
def get_ansatz_ham(params, input_line):    
    """
    Edited for subspace VQE 
    new ansatz builder function using four angles per qubit wire
    params is a list, input_line is an integer (0 to 7)
    This ansatz measures in the P basis (momentum space) with Fourier transform block
    """   

    qr = QuantumRegister(QUBIT_COUNT)        
    cr = ClassicalRegister(QUBIT_COUNT)
    qc = QuantumCircuit(qr, cr)
    
    # input section
    if input_line == 0:       # 000 in computational basis
        pass
    elif input_line == 1:     # 001
        qc.rx(np.pi, qr[0])
    elif input_line == 2:     # 010
        qc.rx(np.pi, qr[1])
    elif input_line == 3:     # 011
        qc.rx(np.pi, qr[0])
        qc.rx(np.pi, qr[1])
    elif input_line == 4:     # 100
        qc.rx(np.pi, qr[2])    
    elif input_line == 5:     # 101
        qc.rx(np.pi, qr[0])
        qc.rx(np.pi, qr[2]) 
    elif input_line == 6:     # 110
        qc.rx(np.pi, qr[1])
        qc.rx(np.pi, qr[2]) 
    elif input_line == 7:     # 111
        qc.rx(np.pi, qr[0])
        qc.rx(np.pi, qr[1])
        qc.rx(np.pi, qr[2])
    else:
        print("Error: input_line should be integer from 0 to 7")
        return
    
    qc.barrier()    # for clarity, after input setup
    
    for layer in range(DEPTH):    
        # pre-entangling section
        qc.ry(params[2*layer], qr[0])    
        qc.rz(params[2*layer + 1], qr[0])
        qc.ry(params[2*layer + 2*DEPTH + 2], qr[1])   # second wire
        qc.rz(params[2*layer + 2*DEPTH + 3], qr[1])    
        if QUBIT_COUNT >= 3:
            qc.ry(params[2*layer + 4*DEPTH + 4], qr[2])   # third wire
            qc.rz(params[2*layer + 4*DEPTH + 5], qr[2])    
        if QUBIT_COUNT >= 4:
            qc.ry(params[2*layer + 6*DEPTH + 6], qr[3])   # fourth wire     
            qc.rz(params[2*layer + 6*DEPTH + 7], qr[3])

        # entangling
        qc.rzx(np.pi/2, qr[0], qr[1])
        if QUBIT_COUNT >= 3:
            qc.rzx(np.pi/2, qr[1], qr[2])
        if QUBIT_COUNT >= 4:
            qc.rzx(np.pi/2, qr[2], qr[3])
        qc.barrier()    
    
    qc.ry(params[2*DEPTH], qr[0])   # first wire 
    qc.rz(params[2*DEPTH+1], qr[0])
    qc.ry(params[4*DEPTH+2], qr[1])   # second wire
    qc.rz(params[4*DEPTH+3], qr[1])
    if QUBIT_COUNT >= 3:
        qc.ry(params[6*DEPTH+4], qr[2])  # third wire
        qc.rz(params[6*DEPTH+5], qr[2])    
    if QUBIT_COUNT >= 4:
        qc.ry(params[8*DEPTH+6], qr[3])   # fourth wire      
        qc.rz(params[8*DEPTH+7], qr[3])

    qubit_list = []
    for i in range(QUBIT_COUNT):
        qubit_list.append(i)
    
    qc.unitary(FourierOperator,qubit_list,label="Fourier")
    for j in range(QUBIT_COUNT):
        qc.measure(qr[j], cr[j])

    return qc

In [ ]:
print("===== Sample get_ansatz_ham circuit for momentum space measurement =====")
my_circuit = get_ansatz_ham(wild_params, 3)
my_circuit.draw(output='mpl')

## 3. Support functions 

(own code) get_output function, get_probability_distribution_new, costArray.


In [ ]:
# result automation
allresults = []   # list
def get_output(dec_number):
    # decimal number is input, output as binary
    # with zeros in front --> 1 becomes 001 when QUBIT_COUNT=3
    # outputs binary as string
    bin_number = str(bin(dec_number).replace("0b", ""))
    while len(bin_number) < QUBIT_COUNT:
        bin_number = "0" + bin_number
    return bin_number

for count in range(n):    # where n is 2**QUBIT_COUNT
    allresults.append(get_output(count))
print(allresults)

In [ ]:
def get_probability_distribution_new(counts):
    # return array of probability    21st May
    output_distr = []    # initialize empty array
    for result in allresults:    # automate this list too
        if counts.get(result) != None:
            output_distr.append(counts.get(result)/NUM_SHOTS)
        else:
            output_distr.append(0)
        #print(result)       
    return output_distr

In [ ]:
costArray = []    # to assign cost for each point on lattice
for point in lattice:
    costArray.append(2*np.pi/(2**QUBIT_COUNT)*point**2/2)
print(costArray)   # pre-factor; not just lattice squared


Finally, we create an instance of the COBYLA optimizer, and run the algorithm. Note that the output varies from run to run. Moreover, while close, the obtained distribution might not be exactly the same as the target distribution, however, increasing the number of shots taken will increase the accuracy of the output.

COBYLA is different from the previous optimizer I used in v1.1. 

## 4. Cost functions to minimize 

Now we specify the objective function which takes as input a list of the variational form's parameters, and returns the cost associated with those parameters:

### new cost function (L1)
Step 2 of algorithm

In [ ]:
def objective_function_weighted_SSQVE(params):
    """
    New objective function for Subspace VQE search, computing sum across different line inputs
    Includes a for loop over ENERGY_STATES constant
    Prints and returns cost value for params
    """
    cost = 0
    #print("params is", params)
    for line in range(ENERGY_STATES):    
        qc_Ham = get_ansatz_ham(params, line)
        qc_Xpos = get_ansatz_pos(params, line)      # potential part
        if line == 0: 
            weight = 3
        elif line == 1: 
            weight = 2
        elif line == 2:
            weight = 1
            
        #weight = ENERGY_STATES - line   # ? line (0,1) have weights (2,1) see Nakanishi paper
        
        # === Hamiltonian part (P measurement) ===
        t_qc_Ham = transpile(qc_Ham, backend)
        qobj_Ham = assemble(t_qc_Ham, shots=NUM_SHOTS)     # run circuit NUM_SHOTS times
        result_Ham = backend.run(qobj_Ham).result()
        output_distr_Ham = get_probability_distribution_new(result_Ham.get_counts(qc_Ham))
    
        for i in range(n):
            cost += (costArray[i]*output_distr_Ham[i])*weight   # see costArray def above
    
        # === Potential well part (X measurement) ===
        t_qc_Xpos = transpile(qc_Xpos, backend)
        qobj_Xpos = assemble(t_qc_Xpos, shots=NUM_SHOTS)     # run circuit NUM_SHOTS times
        result_Xpos = backend.run(qobj_Xpos).result()
        output_distr_Xpos = get_probability_distribution_new(result_Xpos.get_counts(qc_Xpos))
    
        if line == 0:  # ground state
            cost += (output_distr_Xpos[0]*1 + output_distr_Xpos[n-1]*1)*weight 
        elif line == 1:
            cost += (output_distr_Xpos[0]*4 + output_distr_Xpos[n-1]*4)*weight 
        elif line == 2:
            cost += (output_distr_Xpos[0]*9 + output_distr_Xpos[n-1]*9)*weight 
        elif line == 3:
            cost += (output_distr_Xpos[0]*16 + output_distr_Xpos[n-1]*16)*weight     
        
        #print(cost, "   ", line)   # to plot
    print(cost)
    return cost

## 5. Optimization

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize 
use scipy optimizer

In [ ]:
#bnds = ((0, 6.2832),)*PARAM_COUNT   # angle bound between 0 and 2*pi
bnds = []
for item in range(PARAM_COUNT):
    bnds.append((0,6.2832))


In [ ]:
from qiskit.aqua.components.optimizers import COBYLA, SPSA
from qiskit.visualization import plot_histogram

#optimizer = COBYLA(maxiter=2000, tol=.0001)
#optimizer = SPSA(maxiter=4000, save_steps=1, last_avg=1, c0=0.62, c1=0.1, c2=0.602, c3=0.101, c4=0, skip_calibration=False)
optimizer = SPSA(maxiter=4000, save_steps=1, last_avg=1, c0=0.62, c1=0.1, c2=0.602, c3=0.101, c4=0, skip_calibration=False)

start_time = time.time()

# Optimize according to Hamiltonian expectation value 
ret = optimizer.optimize(num_vars=PARAM_COUNT, objective_function=objective_function_weighted_SSQVE, \
                         initial_point=wild_params, variable_bounds=bnds)

#print("Lowest eigenvalue is: ", min_eigenvalue)
print("--- %s seconds ---" % round((time.time() - start_time),1))
gate_params = ret[0]   

In [ ]:
# BFGS implementation
#sol = minimize(objective_function_weighted_SSQVE, wild_params, method='BFGS', jac=None, options={'disp': True})
#gate_params = ret.x


## 6. Results

### 6.1 Momentum space

In [ ]:
def get_energy(params, level):
    """
    Helper function to display momentum space probability distribution
    Prints energy level as computed from optimized ansatz params 
    level refers to the input, mutually orthogonal standard computational basis
    state using X gate(s) as required. 
    If level requested is beyond the ENERGY_STATES considered, returns error message
    """
    if level >= ENERGY_STATES:
        # this wasn't considered in weighting
        print("Error: Energy level requested is too high, check ENERGY_STATES.")
        return
    
    # quantum circuit calculation
    qc_Mom = get_ansatz_ham(params, level)   # with Fourier block at the end
    t_qc_Mom = transpile(qc_Mom, backend)
    qobj_Mom = assemble(t_qc_Mom, shots=NUM_SHOTS)   
    counts_Mom = backend.run(qobj_Mom).result().get_counts(qc_Mom)
    output_distr_Mom = get_probability_distribution_new(counts_Mom)

    this_energy = 0
    for k in range(2**QUBIT_COUNT):
        # compute cost function in momentum space
        this_energy += costArray[k]*output_distr_Mom[k]
    multiple = np.round(this_energy/min_eigenvalue, 3)
    
    print("The", level, "level energy is", this_energy, "multiple of", multiple)        
    #plot_histogram(counts_Mom, color='orange', title="Momentum measure")
    return this_energy


In [ ]:
# display energies computed
print("The minimum eigenvalue is", min_eigenvalue)
get_energy(gate_params, 0)
get_energy(gate_params, 1)
get_energy(gate_params, 2)
get_energy(gate_params, 3)

### Visual check of ground state calculations

In [ ]:
# this part for checking only

#gate_params = ret[0]   # for next time
print("Optimized gate parameters:", gate_params)

check_level = 1

# Ground state expectation energy (by momentum space measurement)
qcMom = get_ansatz_ham(gate_params,check_level)   # with Fourier block at the end
t_qc_Mom = transpile(qcMom, backend)
qobj_Mom = assemble(t_qc_Mom, shots=NUM_SHOTS)   
counts_Mom = backend.run(qobj_Mom).result().get_counts(qcMom)
output_distr_Mom = get_probability_distribution_new(counts_Mom)
#print("===== Ground state momentum circuit below: =====")
#qcMom.draw('mpl')

In [ ]:
# ground state
momError = 0 
for i in range(2**QUBIT_COUNT):
    # compute cost function in momentum space
    momError += costArray[i]*output_distr_Mom[i]
multiple = np.round(momError/min_eigenvalue, 3)
print("Momentum cost is ", momError, " a multiple of ", multiple)        
    
from qiskit.visualization import plot_histogram
plot_histogram(counts_Mom, color='orange', title="Momentum measure")


### 6.3 Position space
plot in blue

In [ ]:
# Obtain the position space output distribution using the final optimized parameters
# corresponds to energy
#gate_params = ret[0]
#print(gate_params)
qcPos = get_ansatz_pos(gate_params,check_level)   # maps phi_j to jth excited state Ej of Hamiltonian
t_qc_Pos = transpile(qcPos, backend)
qobj_Pos = assemble(t_qc_Pos, shots=NUM_SHOTS)   
counts_Pos = backend.run(qobj_Pos).result().get_counts(qcPos)
output_distr_Pos = get_probability_distribution_new(counts_Pos)
print("  ")
#print("Obtained Distribution:", output_distr_Pos)
print("===== Final optimized position circuit below: =====")
#qcPos.draw('mpl')

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts_Pos, color='midnightblue', title="Position measure")